In [5]:
import pickle
ocr_counters = pickle.load( open( "ocr_counters_all.pickle", "rb" ) )
fullstops = pickle.load( open( "fullstops.pickle", "rb" ) )
nyt_ids_all = pickle.load( open( "nyt_ids_all.pickle", "rb" ) )
from application.selective_features import dictionaries_without_features
ocr_counters_no_stops_or_names = dictionaries_without_features(ocr_counters, fullstops)


In [40]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
    
def predict_genders(nyt_ids, ocr_counters, r):

    v = DictVectorizer()
    X = v.fit_transform(ocr_counters)
    y = TfidfTransformer()
    Z = y.fit_transform(X)

    scaled_vsm = Z.toarray()
    try:
        from sklearn.model_selection import train_test_split
    except:
        from sklearn.cross_validation import train_test_split
    from sklearn.linear_model import LogisticRegression
    y_as_list = list(range(len(scaled_vsm)))
    train_size_int = 435
    test_size_int = 100
    X_train, X_test, y_train, y_test = train_test_split(scaled_vsm, y_as_list, train_size=train_size_int, \
                                                        test_size=test_size_int, random_state=r)
    train_labels = []
    # 0 = male here ... eventually I'll need to make this dynamic or external,for now it's set number of each
    for pos in y_train:
        if pos < 414:
            value = 0
        else:
            value = 1
        train_labels.append(value)
    test_labels = []
    for pos in y_test:
        if pos < 414:
            value = 0
        else:
            value = 1
        test_labels.append(value)
    lr = LogisticRegression(class_weight={0:1.21, 1:4.14})
    lr.fit(X_train, train_labels)
    results = lr.predict(X_test)
    probs = lr.predict_proba(X_test)
    result_rows = zip(nyt_ids, results, test_labels, [i[0] for i in probs], [i[1] for i in probs])
    
    score = lr.score(X_test, test_labels)
    y_train = [str(u) for u in y_train]
    y_test = [str(z) for z in y_test]
    a = ', '.join(y_train)
    b = ", ".join(y_test)
    #build main_row
    main_row = [a,b, train_size_int, test_size_int, score]
    #train_ids, test_ids, train_size, test_size, accuracy, stopwords, features, weighting
    
    words = v.vocabulary_.keys()
    labeled_coefs = {}
    for i in words:
        try:
            labeled_coefs[i] = lr.coef_[0][v.vocabulary_[i]]
        except: 
            pass
    coef_tuples = labeled_coefs.items()
    import numpy as np
    odds_f = np.exp([i[1] for i in coef_tuples])
    odds_m = np.exp([i[1]*-1.0 for i in coef_tuples])
    terms = [i[0] for i in coef_tuples]
    coefs = [i[1] for i in coef_tuples]
    coef_rows = zip(terms, coefs, odds_m, odds_f)
    return main_row, result_rows, coef_rows

import sqlite3

def store_results(main_row, result_rows, coef_rows):
    conn = sqlite3.connect('regression_scores.db')
    c = conn.cursor()
    make_main = """CREATE TABLE IF NOT EXISTS main 
    (id INTEGER PRIMARY KEY, train_ids TEXT, test_ids TEXT, train_size INTEGER, 
    test_size INTEGER, accuracy REAL, stopwords TEXT, features TEXT, weighting TEXT)"""
    c.execute(make_main)
    make_results = """CREATE TABLE IF NOT EXISTS results (id INTEGER PRIMARY KEY, 
    main_id INTEGER, nyt_id TEXT, predicted INTEGER, labeled INTEGER, probability_m REAL, probability_f REAL, 
    FOREIGN KEY(main_id) REFERENCES main(id))"""
    c.execute(make_results)
    make_coefficients = """CREATE TABLE IF NOT EXISTS coefficients (id INTEGER PRIMARY KEY, 
    main_id INTEGER, term TEXT, coefficient_score REAL, odds_m REAL, odds_f REAL, 
    FOREIGN KEY(main_id) REFERENCES main(id))"""
    c.execute(make_coefficients)
    
    insert_main = """INSERT INTO main (id, test_ids, train_ids, train_size, test_size, 
    accuracy, stopwords, features, weighting) 
    VALUES (null, ?, ?, ?, ?, ?, ?, ?, ?)"""
    c.execute(insert_main, main_row)
    conn.commit()

    #get id for row you just inserted
    main_id = c.execute("""SELECT id FROM main ORDER BY id DESC""").fetchone()[0]
    insert_result = """INSERT INTO results (id, main_id, nyt_id, predicted, labeled, probability_m, 
    probability_f) 
    VALUES (null, ?, ?, ?, ?, ?, ?)"""
    for result_row in result_rows:
        new_row = [main_id]
        new_row.extend(result_row)
        c.execute(insert_result, new_row)
    conn.commit()
    insert_coef = """INSERT INTO coefficients (id, main_id, term, coefficient_score, odds_m, odds_f) 
    VALUES (null, ?, ?, ?, ?, ?)"""
    
    for coef_row in coef_rows:
        new_c_row = [main_id]
        new_c_row.extend(coef_row)
        c.execute(insert_coef, new_c_row)
    conn.commit()

In [41]:
for z in range(500):
    if z % 10 == 0:
        print(z)
    main_row, result_rows, coef_rows = predict_genders(nyt_ids_all, ocr_counters, z)
    main_row.extend(["none", "term frequencies", "tf-idf"])
    store_results(main_row, result_rows, coef_rows)

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490


In [21]:
print(main_row, result_rows[0], coef_rows[0])

((u'4fc045fc45c1498b0d2162b6', 0, 0, 0.77889788359923384, 0.22110211640076619), (u'stock', -0.040830173400804332, 1.0416751863687461, 0.95999214830678181))


In [42]:
for z in range(500):
    if z % 10 == 0:
        print(z)
    main_row, result_rows, coef_rows = predict_genders(nyt_ids_all, ocr_counters_no_stops_or_names, z)
    main_row.extend(["stops, gender terms, names", "term frequencies", "tf-idf"])
    store_results(main_row, result_rows, coef_rows)

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
